In [38]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os
import pickle
import lasagne

num_epoch = 2
input_shape = (28, 28, 1)
# num_classes = 1000
batch_size = 128
dataset = 'downsampled_imagenet'
file_pattern = './train_data_batch_*'

In [39]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo)
    return dict

In [42]:
def load_databatch(data_folder, idx, img_size=64):
    data_file = os.path.join(data_folder, 'train_data_batch_')

    d = unpickle(data_file + str(idx))
    x = d['data']
    y = d['labels']
    mean_image = d['mean']

    x = x/np.float32(255)
    mean_image = mean_image/np.float32(255)

    # Labels are indexed from 1, shift it so that indexes start at 0
    y = [i-1 for i in y]
    data_size = x.shape[0]

    x -= mean_image

    img_size2 = img_size * img_size

    x = np.dstack((x[:, :img_size2], x[:, img_size2:2*img_size2], x[:, 2*img_size2:]))
    x = x.reshape((x.shape[0], img_size, img_size, 3))

    # create mirrored images
    X_train = x[0:data_size, :, :, :]
    Y_train = y[0:data_size]
    X_train_flip = X_train[:, :, :, ::-1]
    Y_train_flip = Y_train
    X_train = np.concatenate((X_train, X_train_flip), axis=0)
    Y_train = np.concatenate((Y_train, Y_train_flip), axis=0)

    return dict(
        X_train=lasagne.utils.floatX(X_train),
        Y_train=Y_train.astype('int32'),
        mean=mean_image)
    # return (lasagne.utils.floatX(X_train), Y_train.astype('int32'))
    

In [43]:
dataset = load_databatch('./', 1, 64)
X_train = dataset['X_train']
Y_train = dataset['Y_train']
# num_classes = np.unique(Y_train)


In [44]:
print(X_train.shape)

(256232, 64, 64, 3)


In [45]:
from tensorflow.keras.applications import ResNet50
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]))
# Freeze ResNet50 layers
for layer in resnet50.layers:
    layer.trainable = False

In [48]:
# print(num_classes)
model = tf.keras.Sequential([
    resnet50,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1000, activation='softmax')
])

In [49]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [51]:
print((X_train.shape))
print(Y_train.shape)

(256232, 64, 64, 3)
(256232,)


In [50]:
batch_size=128
history = model.fit(X_train, Y_train, epochs=num_epoch, batch_size=batch_size)

Epoch 1/2
2002/2002 [==============================] - 336s 165ms/step - loss: 6.8581 - accuracy: 0.0020
Epoch 2/2
  39/2002 [..............................] - ETA: 4:57 - loss: 6.7820 - accuracy: 0.0038

KeyboardInterrupt: 

In [16]:
X_train = X_train / 255.
X_train = X_train.astype('float32')
from tensorflow.keras.utils import to_categorical
train_label = to_categorical(Y_train)

In [11]:
# def preprocess(image, label):
#     image = tf.cast(image, tf.float32)
#     image = tf.image.resize(image, (48, 48)) # Resize to match ResNet50 input shape
#     image = tf.keras.applications.resnet50.preprocess_input(image)
#     return image, label
def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (32, 32))
    image = tf.image.grayscale_to_rgb(image)  # Convert grayscale to RGB image
    image = tf.keras.applications.resnet50.preprocess_input(image)
    return image, label

In [12]:
dataset_train = dataset.map(preprocess).cache().shuffle(10000).batch(batch_size)

AttributeError: 'tuple' object has no attribute 'map'

In [5]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [8]:
# mnist_train, mnist_val = mnist_dataset['train'], mnist_dataset['test']
mnist_train = mnist_dataset['train']
print(mnist_train)

<_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [32]:

mnist_train = mnist_train.map(preprocess).cache().shuffle(10000).batch(batch_size)
# mnist_val = mnist_val.map(preprocess).cache().batch(batch_size)

In [34]:
model = tf.keras.Sequential([
    resnet50,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [35]:
# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:

history = model.fit(mnist_train, epochs=num_epoch)